# Intro

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from common_imports import *

/home/kimlab1/strokach/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
import functions as fn

In [4]:
common.configure_logging()

In [5]:
%matplotlib inline

# Load data

## Cell info

In [6]:
cell_info = pd.read_csv(
    '../downloads/challenge_data/sanger_molecular_data/'
    'cell_info.csv/cell_info.csv',
    sep=',',
)

In [7]:
cell_info = cell_info.rename(columns={
    'Sanger.Name': 'cell_line',
    'CCLE.Name': 'cell_line_ccle',
    'Alternative.Name': 'cell_line_alternative',
    'Disease.Area': 'cell_line_disease_area',
    'Tissue..General.': 'cell_line_tissue',
    'COSMIC': 'cell_line_cosmic',
})

In [8]:
cell_info.head(2)

,cell_line,cell_line_ccle,cell_line_alternative,cell_line_disease_area,cell_line_tissue,cell_line_cosmic
0,22RV1,22RV1_PROSTATE,22RV1,Urology,male genital system,924100
1,647-V,647V_URINARY_TRACT,647V,Urology,urinary tract,906797


## CSS

In [9]:
css = pd.read_sql_query("""\
SELECT c, avg(synergy_score) synergy_score
FROM az_dream_2015.ALL_TRAINING_DATA_WSYNERGY
where synergy_score is not null
group by c;
""", sa.create_engine('{}/az_dream_2015'.format(os.environ['BIODB_CONNECTION_STR'])))

In [10]:
css = css.set_index('c')['synergy_score']

In [11]:
css.head()

c
22RV1       -8.601674
647-V       13.834063
A549        21.671942
BFTC-905    21.839952
BT-20       22.283808
Name: synergy_score, dtype: float64

## GEx

In [12]:
gex = pd.read_csv(
    '../downloads/challenge_data/sanger_molecular_data/'
    'gex.csv/gex.csv',
    sep=',', low_memory=False,
)

In [13]:
gex = gex.rename(columns={'Unnamed: 0': 'g'}).set_index('g')
gex.columns.names = ['c']

In [14]:
gex.head()

c,22RV1,647-V,A549,BFTC-905,BT-20,BT-474,BT-549,C32,CAL-120,CAL-148,CAL-51,Calu-3,Calu-6,CAMA-1,COLO-205,DMS-114,DU-4475,EVSA-T,HCC1143,HCC1187,HCC1395,HCC1419,HCC1428,HCC1500,HCC1569,HCC1806,HCC1937,HCC1954,HCC38,HCC70,HCT-116,Hs-578-T,HT-1197,HT-1376,HT-29,J82,KATOIII,KMS-11,KU-19-19,LS-513,M14,MCF7,MDA-MB-157,MDA-MB-231,MDA-MB-361,MDA-MB-415,MDA-MB-436,MDA-MB-453,MDA-MB-468,MFM-223,NCI-H1299,NCI-H1563,NCI-H1703,NCI-H1793,NCI-H1975,NCI-H2085,NCI-H2170,NCI-H2228,NCI-H226,NCI-H2291,NCI-H23,NCI-H3122,NCI-H358,NCI-H520,NCI-H522,NCI-H747,NCI-H838,NCI-SNU-16,RKO,RT4,SW48,SW620,SW780,SW837,SW900,SW948,T-24,T47D,TCCSUP,UACC-812,UM-UC-3,VCaP,VM-CUB-1
g,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TSPAN6,4.804487,8.713556,8.495961,7.813436,7.971621,5.178131,6.383148,5.542645,7.632023,8.121539,8.712338,9.041562,7.413802,6.900407,7.065005,7.548671,3.063309,7.411106,7.581938,9.013253,7.435151,3.455951,7.241512,6.981643,7.926592,8.052567,8.787131,8.548208,7.452954,8.431625,7.571165,5.138861,7.079121,6.290094,7.356123,7.381170,7.448062,3.293176,7.791892,8.116502,7.572672,7.625013,5.500209,7.783983,3.285879,6.391809,7.277258,3.366715,8.680800,3.257614,6.893817,8.899760,6.678621,7.267140,7.923672,7.657637,8.535321,7.077455,5.807695,8.238266,7.319616,9.106234,8.578107,7.505294,8.827198,7.684001,7.336914,7.733795,7.043944,10.441413,7.455028,7.822307,9.801944,8.141044,7.755154,7.709982,7.788981,7.342996,7.401340,7.086304,6.410431,5.348023,7.187071
TNMD,2.848128,2.834723,2.886997,2.879330,2.792528,3.015506,2.746627,2.736643,2.964585,2.990916,2.643508,2.802379,2.885299,2.726163,2.712504,2.777716,2.878842,3.096118,2.814303,2.952093,2.832570,3.290184,2.772921,2.972959,2.764477,2.658588,2.673902,2.599607,2.884917,2.754649,2.782633,2.793515,2.823985,2.792935,2.905907,2.660366,2.887007,2.719466,2.626371,2.699422,2.884310,2.906657,3.104776,2.845123,2.911672,2.843846,2.783901,2.957776,3.067673,2.679842,2.741474,2.723766,3.227127,2.889134,2.709755,2.795374,2.676706,2.885422,2.782251,2.873048,2.780444,2.721411,2.862169,2.775341,2.891027,2.605938,2.794700,3.078431,2.912598,2.628488,2.816230,3.061145,2.820406,2.911450,2.816921,3.212680,2.984905,2.819535,2.724977,2.952920,2.795879,2.988278,2.691417
DPM1,10.331994,10.504748,9.643436,10.272719,10.601207,10.548792,10.392289,9.890112,10.379553,10.359588,9.880733,9.974280,10.182692,9.578207,10.363337,11.807341,9.403298,10.058010,10.363326,9.347429,10.344827,11.570155,9.721450,10.482541,10.152998,9.947073,10.488027,10.375991,10.173322,9.317847,10.143152,9.845822,9.505074,9.809464,10.404117,10.843943,10.642938,10.303324,10.682479,9.683543,10.406417,10.064238,10.910371,10.047809,9.884430,10.279155,10.716613,9.484411,10.843286,10.803668,9.878794,9.869868,9.850757,10.311008,10.040750,10.244470,10.311849,9.204721,10.283935,9.156797,10.482441,9.418260,10.150886,9.500202,9.710483,10.315858,10.865909,9.650213,10.592788,10.047345,9.200010,10.005268,10.594632,10.072444,10.214976,10.170757,10.558936,10.786582,9.864280,11.536803,10.326887,9.000925,10.184775
SCYL3,5.145387,3.792188,4.154471,4.050385,5.622674,5.829252,3.802817,3.346422,3.614794,5.680584,3.956230,4.291778,4.093346,4.800802,5.309871,4.066887,6.121481,5.142190,3.770037,4.982836,3.877500,5.560883,6.734927,5.322880,4.968159,3.678567,4.592644,5.178378,3.738263,5.360431,4.166403,3.721672,4.027638,4.152946,4.826544,3.944321,3.636842,4.476928,4.465878,4.825148,3.902865,5.084841,4.481802,4.425095,4.219952,4.043486,4.112699,6.738753,4.847348,4.578945,3.458342,4.051185,4.436888,3.384396,3.647858,4.164730,4.143206,4.256899,3.818307,3.969596,3.734341,4.271343,5.208023,4.766765,3.997366,4.706506,4.271641,4.782889,3.682798,4.641314,4.161932,3.993789,4.468384,4.531813,4.388774,4.606470,3.789149,4.677363,3.843770,4.520248,3.512380,4.306414,3.512961
C1orf112,3.545193,3.946664,3.307310,3.701491,3.740521,3.800657,4.379548,3.067727,3.380681,4.056182,3.236620,3.784500,3.716141,3.240325,3.354821,3.732485,3.384682,3.678305,3.394502,4.122282,3.55565

In [15]:
!ls '../downloads/challenge_data/sanger_molecular_data/gex.csv/'

gex.csv


In [16]:
gex.isnull().any().any()

False

## Gene subsets

### CGS

In [17]:
engine = sa.create_engine('{}/cosmic'.format(os.environ['BIODB_CONNECTION_STR']))
cancer_gene_census = pd.read_sql_table('cancer_gene_census', engine)

In [18]:
cancer_gene_census.head(2)

,gene_symbol,name,entrez_geneid,genome_location,chr_band,somatic,germline,tumour_types_somatic,tumour_types_germline,cancer_syndrome,tissue_type,molecular_genetics,mutation_types,translocation_partner,other_germline_mut,other_syndrome,synonyms
0,ABI1,abl-interactor 1,10006,10:26748570-26860863,10p11.2,yes,None,AML,None,None,L,Dom,T,KMT2A,None,None,"ABI1,E3B1,ABI-1,SSH3BP1,10006"
1,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25,9:130835447-130885683,9q34.1,yes,None,"CML, ALL, T-ALL",None,None,L,Dom,"T, Mis","BCR, ETV6, NUP214",None,None,"ABL1,p150,ABL,c-ABL,JTK7,bcr/abl,v-abl,P00519,..."


In [19]:
print("Total number of genes:", len(set(gex.index)))
print("Number of cancer gene census genes:", len(set(cancer_gene_census['gene_symbol'])))
print("Overlap:", len(set(gex.index) & set(cancer_gene_census['gene_symbol'])))

Total number of genes: 17419
Number of cancer gene census genes: 571
Overlap: 526


In [20]:
cgs_genes = list(set(gex.index) & set(cancer_gene_census['gene_symbol']))

### Targets

In [21]:
engine = sa.create_engine('{}/az_dream_2015'.format(os.environ['BIODB_CONNECTION_STR']))
drug_to_hgnc_target = pd.read_sql_table('drug_to_hgnc_target', engine)
drug_to_hgnc_target.head(2)

,drug,hgnc_name
0,ADAM17,ADAM17
1,AKT,AKT1


In [22]:
print("Total number of genes:", len(set(gex.index)))
print("Number of cancer gene census genes:", len(set(drug_to_hgnc_target['hgnc_name'])))
print("Overlap:", len(set(gex.index) & set(drug_to_hgnc_target['hgnc_name'])))

Total number of genes: 17419
Number of cancer gene census genes: 222
Overlap: 207


In [23]:
target_genes = list(set(gex.index) & set(drug_to_hgnc_target['hgnc_name']))

# Save

In [24]:
import csv2sql
db = csv2sql.DataFrameToMySQL(
    '{}/az_dream_2015_features'.format(os.environ['BIODB_CONNECTION_STR']), 
    'gex', 
    os.environ['STG_SERVER_IP'], 
    echo=False
)

In [25]:
import csv2sql.g2d

## GC

In [26]:
gex.head()

c,22RV1,647-V,A549,BFTC-905,BT-20,BT-474,BT-549,C32,CAL-120,CAL-148,CAL-51,Calu-3,Calu-6,CAMA-1,COLO-205,DMS-114,DU-4475,EVSA-T,HCC1143,HCC1187,HCC1395,HCC1419,HCC1428,HCC1500,HCC1569,HCC1806,HCC1937,HCC1954,HCC38,HCC70,HCT-116,Hs-578-T,HT-1197,HT-1376,HT-29,J82,KATOIII,KMS-11,KU-19-19,LS-513,M14,MCF7,MDA-MB-157,MDA-MB-231,MDA-MB-361,MDA-MB-415,MDA-MB-436,MDA-MB-453,MDA-MB-468,MFM-223,NCI-H1299,NCI-H1563,NCI-H1703,NCI-H1793,NCI-H1975,NCI-H2085,NCI-H2170,NCI-H2228,NCI-H226,NCI-H2291,NCI-H23,NCI-H3122,NCI-H358,NCI-H520,NCI-H522,NCI-H747,NCI-H838,NCI-SNU-16,RKO,RT4,SW48,SW620,SW780,SW837,SW900,SW948,T-24,T47D,TCCSUP,UACC-812,UM-UC-3,VCaP,VM-CUB-1
g,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TSPAN6,4.804487,8.713556,8.495961,7.813436,7.971621,5.178131,6.383148,5.542645,7.632023,8.121539,8.712338,9.041562,7.413802,6.900407,7.065005,7.548671,3.063309,7.411106,7.581938,9.013253,7.435151,3.455951,7.241512,6.981643,7.926592,8.052567,8.787131,8.548208,7.452954,8.431625,7.571165,5.138861,7.079121,6.290094,7.356123,7.381170,7.448062,3.293176,7.791892,8.116502,7.572672,7.625013,5.500209,7.783983,3.285879,6.391809,7.277258,3.366715,8.680800,3.257614,6.893817,8.899760,6.678621,7.267140,7.923672,7.657637,8.535321,7.077455,5.807695,8.238266,7.319616,9.106234,8.578107,7.505294,8.827198,7.684001,7.336914,7.733795,7.043944,10.441413,7.455028,7.822307,9.801944,8.141044,7.755154,7.709982,7.788981,7.342996,7.401340,7.086304,6.410431,5.348023,7.187071
TNMD,2.848128,2.834723,2.886997,2.879330,2.792528,3.015506,2.746627,2.736643,2.964585,2.990916,2.643508,2.802379,2.885299,2.726163,2.712504,2.777716,2.878842,3.096118,2.814303,2.952093,2.832570,3.290184,2.772921,2.972959,2.764477,2.658588,2.673902,2.599607,2.884917,2.754649,2.782633,2.793515,2.823985,2.792935,2.905907,2.660366,2.887007,2.719466,2.626371,2.699422,2.884310,2.906657,3.104776,2.845123,2.911672,2.843846,2.783901,2.957776,3.067673,2.679842,2.741474,2.723766,3.227127,2.889134,2.709755,2.795374,2.676706,2.885422,2.782251,2.873048,2.780444,2.721411,2.862169,2.775341,2.891027,2.605938,2.794700,3.078431,2.912598,2.628488,2.816230,3.061145,2.820406,2.911450,2.816921,3.212680,2.984905,2.819535,2.724977,2.952920,2.795879,2.988278,2.691417
DPM1,10.331994,10.504748,9.643436,10.272719,10.601207,10.548792,10.392289,9.890112,10.379553,10.359588,9.880733,9.974280,10.182692,9.578207,10.363337,11.807341,9.403298,10.058010,10.363326,9.347429,10.344827,11.570155,9.721450,10.482541,10.152998,9.947073,10.488027,10.375991,10.173322,9.317847,10.143152,9.845822,9.505074,9.809464,10.404117,10.843943,10.642938,10.303324,10.682479,9.683543,10.406417,10.064238,10.910371,10.047809,9.884430,10.279155,10.716613,9.484411,10.843286,10.803668,9.878794,9.869868,9.850757,10.311008,10.040750,10.244470,10.311849,9.204721,10.283935,9.156797,10.482441,9.418260,10.150886,9.500202,9.710483,10.315858,10.865909,9.650213,10.592788,10.047345,9.200010,10.005268,10.594632,10.072444,10.214976,10.170757,10.558936,10.786582,9.864280,11.536803,10.326887,9.000925,10.184775
SCYL3,5.145387,3.792188,4.154471,4.050385,5.622674,5.829252,3.802817,3.346422,3.614794,5.680584,3.956230,4.291778,4.093346,4.800802,5.309871,4.066887,6.121481,5.142190,3.770037,4.982836,3.877500,5.560883,6.734927,5.322880,4.968159,3.678567,4.592644,5.178378,3.738263,5.360431,4.166403,3.721672,4.027638,4.152946,4.826544,3.944321,3.636842,4.476928,4.465878,4.825148,3.902865,5.084841,4.481802,4.425095,4.219952,4.043486,4.112699,6.738753,4.847348,4.578945,3.458342,4.051185,4.436888,3.384396,3.647858,4.164730,4.143206,4.256899,3.818307,3.969596,3.734341,4.271343,5.208023,4.766765,3.997366,4.706506,4.271641,4.782889,3.682798,4.641314,4.161932,3.993789,4.468384,4.531813,4.388774,4.606470,3.789149,4.677363,3.843770,4.520248,3.512380,4.306414,3.512961
C1orf112,3.545193,3.946664,3.307310,3.701491,3.740521,3.800657,4.379548,3.067727,3.380681,4.056182,3.236620,3.784500,3.716141,3.240325,3.354821,3.732485,3.384682,3.678305,3.394502,4.122282,3.55565

In [27]:
gex_gbgc = (
    gex
    .unstack()
    .reset_index()
    .reindex_axis(['g', 'c', 0], axis=1)
    .rename(columns={0: 'gex_gbgc'})
)
gex_gbgc.head()

,g,c,gex_gbgc
0,TSPAN6,22RV1,4.804487
1,TNMD,22RV1,2.848128
2,DPM1,22RV1,10.331994
3,SCYL3,22RV1,5.145387
4,C1orf112,22RV1,3.545193


In [28]:
db.import_table(
    gex_gbgc, 
    'gex_gbgc', [
        [('g', 'c'), True],
        [('c', 'g'), False],
    ],
)

In [29]:
csv2sql.g2d.g2d('gex_gbgc')

Unknown ending for column: gex_gbgc. Assuming mean...
DROP TABLE IF EXISTS az_dream_2015_features.gex_gbdc;

CREATE TABLE az_dream_2015_features.gex_gbdc AS
SELECT
d2t.drug d,
t.c c,
avg(t.gex_gbgc) gex_gbgc_mean
FROM az_dream_2015.drug_to_hgnc_target d2t
LEFT JOIN az_dream_2015_features.gex_gbgc t ON (t.g = d2t.hgnc_name)
GROUP BY d2t.drug , t.c;

ALTER TABLE az_dream_2015_features.gex_gbdc
MODIFY d VARCHAR(255);

CREATE INDEX a ON az_dream_2015_features.gex_gbdc (d);

DROP INDEX a ON az_dream_2015_features.gex_gbdc;

ALTER TABLE az_dream_2015_features.gex_gbdc
MODIFY c VARCHAR(255);

CREATE INDEX a ON az_dream_2015_features.gex_gbdc (d, c);

CREATE INDEX b ON az_dream_2015_features.gex_gbdc (c, d);

Unknown ending for column: gex_gbgc. Assuming mean...
DROP TABLE IF EXISTS az_dream_2015_features.gex_gbdc_stitch;

CREATE TABLE az_dream_2015_features.gex_gbdc_stitch AS
SELECT
d2t.drug d,
t.c c,
avg(t.gex_gbgc) gex_gbgc_stitch_mean
FROM az_dream_2015.drug_to_hgnc_target_stitch d2t
LEFT 

## G

In [30]:
gex_gbg = (
    gex
    .unstack()
    .reset_index()
    .rename(columns={0: 'gex_gbg'})
    .groupby('g')
    .agg(['max', 'min', 'mean', 'std'])
    .reset_index()
)
gex_gbg.columns = ['_'.join(c).strip('_') for c in gex_gbg.columns]
gex_gbg.head()

,g,gex_gbg_max,gex_gbg_min,gex_gbg_mean,gex_gbg_std
0,A1BG,7.557265,2.733173,4.128894,0.997552
1,A1CF,7.616778,2.807124,3.264452,0.900529
2,A2M,12.962624,3.015057,3.900166,1.688586
3,A2ML1,5.398271,2.618580,2.959910,0.351209
4,A3GALT2P,3.350766,2.460609,2.836749,0.150677


In [33]:
db.import_table(
    gex_gbg, 
    'gex_gbg', [
        [('g', ), True],
    ],
)

INFO:csv2sql.core:bzip2 -dkf '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/gex/gex_gbg.tsv.bz2'
INFO:csv2sql.core:mysql  az_dream_2015_features -e "load data local infile '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/gex/gex_gbg.tsv' into table `gex_gbg` ignore 1 lines;  show warnings;" 
INFO:csv2sql.core:rm -f '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/gex/gex_gbg.tsv'


In [34]:
csv2sql.g2d.g2d('gex_gbg')

DROP TABLE IF EXISTS az_dream_2015_features.gex_gbd;

CREATE TABLE az_dream_2015_features.gex_gbd AS
SELECT
d2t.drug d,
max(t.gex_gbg_max) gex_gbg_max_max,
min(t.gex_gbg_min) gex_gbg_min_min,
avg(t.gex_gbg_mean) gex_gbg_mean_mean,
avg(t.gex_gbg_std) gex_gbg_std_mean
FROM az_dream_2015.drug_to_hgnc_target d2t
LEFT JOIN az_dream_2015_features.gex_gbg t ON (t.g = d2t.hgnc_name)
GROUP BY d2t.drug;

ALTER TABLE az_dream_2015_features.gex_gbd
MODIFY d VARCHAR(255);

CREATE INDEX a ON az_dream_2015_features.gex_gbd (d);

DROP TABLE IF EXISTS az_dream_2015_features.gex_gbd_stitch;

CREATE TABLE az_dream_2015_features.gex_gbd_stitch AS
SELECT
d2t.drug d,
max(t.gex_gbg_max) gex_gbg_max_stitch_max,
min(t.gex_gbg_min) gex_gbg_min_stitch_min,
avg(t.gex_gbg_mean) gex_gbg_mean_stitch_mean,
avg(t.gex_gbg_std) gex_gbg_std_stitch_mean
FROM az_dream_2015.drug_to_hgnc_target_stitch d2t
LEFT JOIN az_dream_2015_features.gex_gbg t ON (t.g = d2t.hgnc_name)
GROUP BY d2t.drug;

ALTER TABLE az_dream_2015_feature

## C

In [35]:
gex_gbc = (
    gex
    .unstack()
    .reset_index()
    .rename(columns={0: 'gex_gbc'})
    .groupby('c')
    .agg(['max', 'min', 'mean', 'std'])
    .reset_index()
)
gex_gbc.columns = ['_'.join(c).strip('_') for c in gex_gbc.columns]
gex_gbc.head()

,c,gex_gbc_max,gex_gbc_min,gex_gbc_mean,gex_gbc_std
0,22RV1,13.480581,2.422259,4.834875,2.128317
1,647-V,13.223673,2.171851,4.815863,2.191772
2,A549,13.279687,2.378473,4.833703,2.158068
3,BFTC-905,13.414020,2.390549,4.846585,2.195542
4,BT-20,13.232747,2.315347,4.833945,2.158957


In [36]:
gex_sscgs_gbc = (
    gex
    .loc[cgs_genes, :]
    .unstack()
    .reset_index()
    .rename(columns={0: 'gex_sscgs_gbc'})
    .groupby('c')
    .agg(['max', 'min', 'mean', 'std'])
    .reset_index()
)
gex_sscgs_gbc.columns = ['_'.join(c).strip('_') for c in gex_sscgs_gbc.columns]
gex_sscgs_gbc.head()

,c,gex_sscgs_gbc_max,gex_sscgs_gbc_min,gex_sscgs_gbc_mean,gex_sscgs_gbc_std
0,22RV1,12.672391,2.622300,5.299643,2.237300
1,647-V,12.303781,2.497122,5.414134,2.292529
2,A549,12.268029,2.541537,5.207579,2.174302
3,BFTC-905,12.679027,2.516691,5.356758,2.283325
4,BT-20,12.518606,2.450278,5.369467,2.245110


In [37]:
gex_sstarget_gbc = (
    gex
    .loc[target_genes, :]
    .unstack()
    .reset_index()
    .rename(columns={0: 'gex_sstarget_gbc'})
    .groupby('c')
    .agg(['max', 'min', 'mean', 'std'])
    .reset_index()
)
gex_sstarget_gbc.columns = ['_'.join(c).strip('_') for c in gex_sstarget_gbc.columns]
gex_sstarget_gbc.head()

,c,gex_sstarget_gbc_max,gex_sstarget_gbc_min,gex_sstarget_gbc_mean,gex_sstarget_gbc_std
0,22RV1,11.925698,2.730148,5.687632,2.540180
1,647-V,11.811346,2.688504,6.107478,2.706002
2,A549,12.931480,2.687166,5.939549,2.578418
3,BFTC-905,13.021741,2.676968,5.978464,2.706339
4,BT-20,13.112763,2.599357,5.985670,2.670173


In [38]:
gex_gbc_all = gex_gbc.merge(gex_sscgs_gbc, on='c', how='left').merge(gex_sstarget_gbc, on='c', how='left')

In [39]:
gex_gbc_all.head()

,c,gex_gbc_max,gex_gbc_min,gex_gbc_mean,gex_gbc_std,gex_sscgs_gbc_max,gex_sscgs_gbc_min,gex_sscgs_gbc_mean,gex_sscgs_gbc_std,gex_sstarget_gbc_max,gex_sstarget_gbc_min,gex_sstarget_gbc_mean,gex_sstarget_gbc_std
0,22RV1,13.480581,2.422259,4.834875,2.128317,12.672391,2.622300,5.299643,2.237300,11.925698,2.730148,5.687632,2.540180
1,647-V,13.223673,2.171851,4.815863,2.191772,12.303781,2.497122,5.414134,2.292529,11.811346,2.688504,6.107478,2.706002
2,A549,13.279687,2.378473,4.833703,2.158068,12.268029,2.541537,5.207579,2.174302,12.931480,2.687166,5.939549,2.578418
3,BFTC-905,13.414020,2.390549,4.846585,2.195542,12.679027,2.516691,5.356758,2.283325,13.021741,2.676968,5.978464,2.706339
4,BT-20,13.232747,2.315347,4.833945,2.158957,12.518606,2.450278,5.369467,2.245110,13.112763,2.599357,5.985670,2.670173


In [40]:
db.import_table(
    gex_gbc_all, 
    'gex_gbc', [
        [('c', ), True],
    ],
)

INFO:csv2sql.core:bzip2 -dkf '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/gex/gex_gbc.tsv.bz2'
INFO:csv2sql.core:mysql  az_dream_2015_features -e "load data local infile '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/gex/gex_gbc.tsv' into table `gex_gbc` ignore 1 lines;  show warnings;" 
INFO:csv2sql.core:rm -f '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/gex/gex_gbc.tsv'


## CC

In [41]:
pa = pd.Panel({'gex': gex})

In [42]:
pa['gex', :, :].head()

c,22RV1,647-V,A549,BFTC-905,BT-20,BT-474,BT-549,C32,CAL-120,CAL-148,CAL-51,Calu-3,Calu-6,CAMA-1,COLO-205,DMS-114,DU-4475,EVSA-T,HCC1143,HCC1187,HCC1395,HCC1419,HCC1428,HCC1500,HCC1569,HCC1806,HCC1937,HCC1954,HCC38,HCC70,HCT-116,Hs-578-T,HT-1197,HT-1376,HT-29,J82,KATOIII,KMS-11,KU-19-19,LS-513,M14,MCF7,MDA-MB-157,MDA-MB-231,MDA-MB-361,MDA-MB-415,MDA-MB-436,MDA-MB-453,MDA-MB-468,MFM-223,NCI-H1299,NCI-H1563,NCI-H1703,NCI-H1793,NCI-H1975,NCI-H2085,NCI-H2170,NCI-H2228,NCI-H226,NCI-H2291,NCI-H23,NCI-H3122,NCI-H358,NCI-H520,NCI-H522,NCI-H747,NCI-H838,NCI-SNU-16,RKO,RT4,SW48,SW620,SW780,SW837,SW900,SW948,T-24,T47D,TCCSUP,UACC-812,UM-UC-3,VCaP,VM-CUB-1
g,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TSPAN6,4.804487,8.713556,8.495961,7.813436,7.971621,5.178131,6.383148,5.542645,7.632023,8.121539,8.712338,9.041562,7.413802,6.900407,7.065005,7.548671,3.063309,7.411106,7.581938,9.013253,7.435151,3.455951,7.241512,6.981643,7.926592,8.052567,8.787131,8.548208,7.452954,8.431625,7.571165,5.138861,7.079121,6.290094,7.356123,7.381170,7.448062,3.293176,7.791892,8.116502,7.572672,7.625013,5.500209,7.783983,3.285879,6.391809,7.277258,3.366715,8.680800,3.257614,6.893817,8.899760,6.678621,7.267140,7.923672,7.657637,8.535321,7.077455,5.807695,8.238266,7.319616,9.106234,8.578107,7.505294,8.827198,7.684001,7.336914,7.733795,7.043944,10.441413,7.455028,7.822307,9.801944,8.141044,7.755154,7.709982,7.788981,7.342996,7.401340,7.086304,6.410431,5.348023,7.187071
TNMD,2.848128,2.834723,2.886997,2.879330,2.792528,3.015506,2.746627,2.736643,2.964585,2.990916,2.643508,2.802379,2.885299,2.726163,2.712504,2.777716,2.878842,3.096118,2.814303,2.952093,2.832570,3.290184,2.772921,2.972959,2.764477,2.658588,2.673902,2.599607,2.884917,2.754649,2.782633,2.793515,2.823985,2.792935,2.905907,2.660366,2.887007,2.719466,2.626371,2.699422,2.884310,2.906657,3.104776,2.845123,2.911672,2.843846,2.783901,2.957776,3.067673,2.679842,2.741474,2.723766,3.227127,2.889134,2.709755,2.795374,2.676706,2.885422,2.782251,2.873048,2.780444,2.721411,2.862169,2.775341,2.891027,2.605938,2.794700,3.078431,2.912598,2.628488,2.816230,3.061145,2.820406,2.911450,2.816921,3.212680,2.984905,2.819535,2.724977,2.952920,2.795879,2.988278,2.691417
DPM1,10.331994,10.504748,9.643436,10.272719,10.601207,10.548792,10.392289,9.890112,10.379553,10.359588,9.880733,9.974280,10.182692,9.578207,10.363337,11.807341,9.403298,10.058010,10.363326,9.347429,10.344827,11.570155,9.721450,10.482541,10.152998,9.947073,10.488027,10.375991,10.173322,9.317847,10.143152,9.845822,9.505074,9.809464,10.404117,10.843943,10.642938,10.303324,10.682479,9.683543,10.406417,10.064238,10.910371,10.047809,9.884430,10.279155,10.716613,9.484411,10.843286,10.803668,9.878794,9.869868,9.850757,10.311008,10.040750,10.244470,10.311849,9.204721,10.283935,9.156797,10.482441,9.418260,10.150886,9.500202,9.710483,10.315858,10.865909,9.650213,10.592788,10.047345,9.200010,10.005268,10.594632,10.072444,10.214976,10.170757,10.558936,10.786582,9.864280,11.536803,10.326887,9.000925,10.184775
SCYL3,5.145387,3.792188,4.154471,4.050385,5.622674,5.829252,3.802817,3.346422,3.614794,5.680584,3.956230,4.291778,4.093346,4.800802,5.309871,4.066887,6.121481,5.142190,3.770037,4.982836,3.877500,5.560883,6.734927,5.322880,4.968159,3.678567,4.592644,5.178378,3.738263,5.360431,4.166403,3.721672,4.027638,4.152946,4.826544,3.944321,3.636842,4.476928,4.465878,4.825148,3.902865,5.084841,4.481802,4.425095,4.219952,4.043486,4.112699,6.738753,4.847348,4.578945,3.458342,4.051185,4.436888,3.384396,3.647858,4.164730,4.143206,4.256899,3.818307,3.969596,3.734341,4.271343,5.208023,4.766765,3.997366,4.706506,4.271641,4.782889,3.682798,4.641314,4.161932,3.993789,4.468384,4.531813,4.388774,4.606470,3.789149,4.677363,3.843770,4.520248,3.512380,4.306414,3.512961
C1orf112,3.545193,3.946664,3.307310,3.701491,3.740521,3.800657,4.379548,3.067727,3.380681,4.056182,3.236620,3.784500,3.716141,3.240325,3.354821,3.732485,3.384682,3.678305,3.394502,4.122282,3.55565

In [43]:
row_subsets = {
    '': None,
    'cgs': cgs_genes,
    'target': target_genes,    
}
cc_distances = fn.get_cc_distance(gex, css, row_subsets=row_subsets)

column_subsets: {'': None}


/home/kimlab1/strokach/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [44]:
cc_distances

,metric,corr,pvalue,count,PW,feature_subset_name,row_subset_name,corr_abs
4,sqeuclidean,-0.060259,6.529012e-07,6806,x y _sqeuclidean scor...,,,0.060259
9,chebyshev,0.162171,2.492046e-41,6806,x y _sscgs_chebyshev ...,,cgs,0.162171
2,cityblock,-0.038708,1.403194e-03,6806,x y _sstarget_citybloc...,,target,0.038708


In [45]:
gex_gbcc = (
    cc_distances.iloc[0]['PW']
    .drop(pd.Index(['score_diff']), axis=1)
    .rename(columns={'x': 'c_x', 'y': 'c_y'})
)
display(gex_gbcc.head())

for i in range(1, len(cc_distances)):
    df_other = (
        cc_distances.iloc[i]['PW']
        .drop(pd.Index(['score_diff']), axis=1)
        .rename(columns={'x': 'c_x', 'y': 'c_y'})
    )   
    gex_gbcc = gex_gbcc.merge(df_other, on=['c_x', 'c_y'], how='left')
display(gex_gbcc.head())

,c_x,c_y,_sqeuclidean
1,647-V,22RV1,20767.375704
2,A549,22RV1,22500.992568
3,BFTC-905,22RV1,21825.566474
4,BT-20,22RV1,22976.099706
5,BT-474,22RV1,19982.372994


,c_x,c_y,_sqeuclidean,_sscgs_chebyshev,_sstarget_cityblock
0,647-V,22RV1,20767.375704,5.629813,155.889332
1,A549,22RV1,22500.992568,5.416149,152.385171
2,BFTC-905,22RV1,21825.566474,5.285617,154.198977
3,BT-20,22RV1,22976.099706,5.091836,152.146012
4,BT-474,22RV1,19982.372994,5.556663,142.576085


In [47]:
db.import_table(
    gex_gbcc, 
    'gex_gbcc', [
        [('c_x', 'c_y', ), True],
    ],
)

INFO:csv2sql.core:bzip2 -dkf '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/gex/gex_gbcc.tsv.bz2'
INFO:csv2sql.core:mysql  az_dream_2015_features -e "load data local infile '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/gex/gex_gbcc.tsv' into table `gex_gbcc` ignore 1 lines;  show warnings;" 
INFO:csv2sql.core:rm -f '/home/kimlab1/database_data/biodata/recipes/az_dream_2015/notebooks/gex/gex_gbcc.tsv'
